# Creating Postgres Users

### Introduction

Roles are a pretty critical feature of postgres and databases in general.  Postgres gives us the ability to control the ability to read or write to databases, tables, or even individual rows in a table.  Let's get started.

### Viewing Users

Whenever we log into or access data from postgres, we are doing so as a role.  It is through a role that we have permission to connect to and access information in a database.  We can switch our roles, just like we can login or out of a website.  In postgres, roles and users are aliases for each other, with the exception that users have the ability to login.

1. The current user

Whenever we are in the postgres shell, we have logged in as a user.  We can view the current user with the `SELECT current_user;` query:

```psql
postgres=# SELECT current_user;
 current_user
--------------
 postgres
(1 row)
```

So we can see that the current user is postgres.  Another way that we can view the current user is with the `/conninfo` command.

```
postgres=# \conninfo
You are connected to database "postgres" as user "postgres" via socket in "/tmp" at port "5432".
```

2. Viewing all users

Of course, our postgres database may have other users created -- each with their own set of permissions that it keeps track of.  We can see a list of roles in our postgres application with the `\du` command, which stands for display users:

<img src="./postgres-roles.png" width="80%">

There we see a list of all roles in our postgres application, and the related permissions of the roles.  Notice that there we see the `postgres` role, who we are logged in as now, and that postgres role has Superuser rights, which also gives them the rights to create other users.

### Changing Users

Now that we know how to view users in postgres, let's create a new role, and give that role superuser rights.  

> We can only issue superuser rights if we are logged in as a superuser, which as the postgres user, we are.

We can create a new role with the `CREATE ROLE` command.

<img src="./create-new-user.png" width="80%">

So we just created a role called `sample_user` with the CREATE ROLE command and set the login password to sample, and issued SUPERUSER rights.  

Then we can change to the `sample_user` from our current session with something like the following:

<img src="./changing-users.png" width="70%">

> The above command said to connect to the current database as the `sample_user`.  Because the current database is `postgres` we are connected to postgres as `sample_user`.

At this point we may notice something odd, which is that we can switch users without typing in a password.  For example, we can change 

The reason why we did not need a password is because we did not yet password protect the `postgres` user.  

> If the postgres user is unprotected, and the user has superuser rights, then there's no use in password protecting other users.

So we can add a password to our postgres user with the ALTER USER command like so:

> `ALTER ROLE postgres WITH PASSWORD 'postgres';`

So now did we password protect our roles?  Well, not quite.  To do that, we'll also have to update our configuration file.  We'll discuss that in the next section.

### Changing our Configuration

So we just created a new user with the CREATE ROLE and updated the postgres role to have a password with the ALTER ROLE command.  Still, we can currently login to either role without a password.  Let's see why.

The issue is that the `pg_hba.conf` file controls whether to require logging in as passwords, and by default this file sets the login policy to trust - as in just trust that an individual is that role, without requiring a login.  

We can change this by first finding and then altering the `pg_hba.conf` file.  We can find configuration files, while logged into postgres with the following:

<img src="./show-conf-file.png" width="70%">

Our pg_hba.conf file should be located in the same folder as `postgresql.conf`.  So let's navigate to that folder.

`cd /Users/jeff/Library/Application\ Support/Postgres/var-11/`

And once there, we can find the `postgresql.conf` file.  At the bottom of that file, we'll see each of the policies set to trust.  To change this, I altered my file to change the METHOD from `trust` to `md5`:

<img src="./change-md5.png">

Then shutdown postgres.

> If you have the postgres app, this can be done by clicking on the app icon, and selecting `quit`.  Then restart the app.

### Logging in Again

This time when we connect to the psql shell, we'll need to provide a password.  We can specify the user we are connecting with by providing the `-U` flag.

<img src="./psql-login.png" width="70%">

Then we can confirm that this worked with the `conninfo` command.

```SQL
postgres=# \conninfo
You are connected to database "postgres" as user "postgres" via socket in "/tmp" at port "5432".
```

Or we can also select the current user.

```SQL
postgres=# SELECT current_user;
 current_user
--------------
 postgres
(1 row)
```

And we can to a different sample_user with `\c - sample_user` command, which this time is password protected.

```SQL
postgres=# SELECT current_user;
 current_user
--------------
 sample_user
(1 row)
```

### Summary

In this section we learned multiple command for viewing and managing roles in postgres.  In doing so, we learned of the following:  

1. Viewing current user
* `\conninfo`
* `SELECT current_user;`

2. Viewing all roles
* `\du`

3. Creating and Altering roles

* CREATE ROLE sample_user LOGIN PASSWORD 'sample' SUPERUSER;
* ALTER ROLE postgres WITH PASSWORD 'postgres';

4. Changing the role

* `\c - sample_user`: connect to the current database as the sample user
* `psql -U sample_user` connect to postgres as the specified user

Then we also learned how to setup our postgres application to enforce logins with passwords.  To do so, we first identified the location of our configuration with the command:

`show config_file;`

Then we navigated to the `pg_hba.conf` and changed the METHOD trust to `md5`.  Then we restarted postgres and authenticated with our password.

### Resources


[User Permissions](https://flaviocopes.com/postgres-user-permissions/)

[Aiven Postgres Intro](https://aiven.io/blog/an-introduction-to-postgresql)

[AWS Blog Postgres Users](https://aws.amazon.com/blogs/database/managing-postgresql-users-and-roles/)